# MCMC Loops for S I Lines
Now that we have created a manual process to input each line and recieve a brightness, I have created a loop that will go through the wavelength list for all S I lines and return the brightness mean, standard deviation, the gaussian width, uncertainty, the line center, and the line center uncertainty, and the given line.

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
from astropy.table import Table
import emcee
from IPython.display import display, Math
from tqdm.auto import tqdm
#%matplotlib inline
%config InlineBackend.figure_format='retina'

In [10]:
line = 1250.8140
hdulist = fits.open('../data/composite_Io_eclipsed.fits')
evt_data = Table(hdulist[1].data)
image_data = hdulist[1].data
df = pd.read_csv('../data/Io_Master_Line_List (1).csv') #spaces in csv file will make df object type and not float64
line = "S I"
target_mask = df.Species.str.endswith(line) #probably a better function but idk it right now
df = df[target_mask]
df_wavelength = df['Wavelength']
df_species = df['Species']

In [11]:
def log_likelihood(theta):
    A, mu, logw = theta
    model = generative_model(A, mu, logw, int_wl = line)
    residual = flux - model
    chi_squared = np.sum(residual** 2 / unc**2)
    return -0.5 * chi_squared

In [13]:
n_walkers = 32
n_params = 3
n_steps = 5000
labels = ["A", "mu", "w"]

In [14]:
df_results = pd.DataFrame()
problem_files = {}

In [15]:
wavelength_list = df_wavelength.values.tolist()

In [16]:
for index in tqdm(range(0, 5)):
    #len(wavelength_list) for whole list
    
    line = wavelength_list[index]
    
    left_bound = 0.999*line
    right_bound = 1.001*line
        
    sub_region = (evt_data['WAVELENGTH'] > left_bound) & (evt_data['WAVELENGTH'] < right_bound) #only take values within this area
    wl = evt_data['WAVELENGTH'][sub_region]
    flux = evt_data['FLUX'][sub_region]
    unc = 0.05*flux #placeholder uncertainty
    
    def generative_model(A, mu, logw, int_wl = line):
        """Generate the model given parameters"""
        w = np.exp(logw)
        gaussian = A * np.exp(-0.5*(wl-mu)**2/w**2)
        return gaussian
    
    A_guess, mu_guess, logw_guess = 0.04*10**-13, line, np.log(0.2)
    flux_guess = generative_model(A_guess, mu_guess, logw_guess)
    
    theta_guess = np.array([A_guess, mu_guess, logw_guess])
    log_likelihood(theta_guess) #more magnitude the log_likelihood, the worse the fit
    
    pos = theta_guess + 1e-4 * np.random.randn(n_walkers, n_params) #intial guess position

    #with Pool() as pool:
    sampler = emcee.EnsembleSampler(n_walkers, n_params, log_likelihood, threads=12)
    sampler.run_mcmc(pos, n_steps, progress=False);

    flat_samples = sampler.get_chain(discard=1000, thin=15, flat=True)

    A_draws = flat_samples[:,0]
    mu_draws = flat_samples[:,1]
    w_draws = np.exp(flat_samples[:, 2])
    
    brightness = -((2*np.pi)**.5)*(A_draws*w_draws)
    brightness
    
    brightness_mean = np.mean(brightness)
    brightness_std = np.std(brightness)
    
    gauss_width = np.mean(w_draws)
    gauss_width_unc = np.std(w_draws)
    obs_line_center = np.mean(mu_draws)
    obs_line_center_unc = np.std(mu_draws)
    
    temp = {'brightness':brightness_mean, 'brightness_unc':brightness_std, 'int_wv':line, 'gaussian_width':gauss_width, 'gaussian_width_unc':gauss_width_unc,
                'obs_line_center':obs_line_center, 'obs_line_center_unc':obs_line_center_unc}
    
    df_results = df_results.append(temp, ignore_index=True)
    if (index % 10) == 0:
            #print(index, fn[-49:])
            df_results.to_csv('../data/io_preliminary_results_2.csv',index=False)

  0%|          | 0/5 [00:00<?, ?it/s]

Now that we have run the loop, let us save the file for later use.

In [18]:
df_results

,brightness,brightness_unc,gaussian_width,gaussian_width_unc,int_wv,obs_line_center,obs_line_center_unc
0,9.567195e-19,5.050943e-20,0.182068,0.004037,1208.850,1208.986858,0.009198
1,7.399353e-18,2.736936e-18,0.083925,0.030241,1211.212,1210.706342,0.259752
2,-3.430411e-18,9.285372e-20,0.053892,0.001337,1211.380,1211.368663,0.002410
3,-1.415385e-17,2.320230e-19,0.049160,0.000600,1212.795,1212.915104,0.000826
4,-9.399315e-18,1.625187e-19,0.035978,0.000423,1218.595,1218.616261,0.000504


In [17]:
df_results.to_csv('../data/io_preliminary_results_2.csv',index=False)

PermissionError: [Errno 13] Permission denied: '../data/io_preliminary_results_2.csv'